In [1]:
import requests
import time
import tkinter as tk
from tkinter import messagebox

In [2]:
API_KEY = '1WVIFMGMYE4TIL72'

class Stock:
    def __init__(self, ticker, quantity, purchase_price):
        self.ticker = ticker.upper()
        self.quantity = quantity
        self.purchase_price = purchase_price
        self.current_price = None

    def fetch_current_price(self):
        try:
            url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={self.ticker}&interval=5min&apikey={API_KEY}'
            response = requests.get(url)
            data = response.json()
            time_series = data.get("Time Series (5min)", None)
            if time_series:
                latest_time = list(time_series.keys())[0]
                latest_data = time_series[latest_time]
                self.current_price = float(latest_data['4. close'])
            else:
                raise ValueError("Error fetching price data from API.")
        except Exception as e:
            print(f"Error fetching data for {self.ticker}: {e}")
            self.current_price = None

    def current_value(self):
        if self.current_price is None:
            self.fetch_current_price()
        return self.quantity * self.current_price if self.current_price else 0

    def profit_loss(self):
        if self.current_price is None:
            self.fetch_current_price()
        return (self.current_price - self.purchase_price) * self.quantity if self.current_price else 0

    def percentage_change(self):
        if self.current_price is None:
            self.fetch_current_price()
        return ((self.current_price - self.purchase_price) / self.purchase_price) * 100 if self.current_price else 0

In [3]:
class Portfolio:
    def __init__(self):
        self.stocks = []

    def add_stock(self, ticker, quantity, purchase_price):
        stock = Stock(ticker, quantity, purchase_price)
        self.stocks.append(stock)

    def remove_stock(self, ticker):
        self.stocks = [stock for stock in self.stocks if stock.ticker != ticker.upper()]

    def portfolio_value(self):
        return sum(stock.current_value() for stock in self.stocks)

    def total_profit_loss(self):
        return sum(stock.profit_loss() for stock in self.stocks)

    def portfolio_summary(self):
        summary = []
        for stock in self.stocks:
            stock.fetch_current_price()
            time.sleep(12)  # To prevent API rate limit issues
            summary.append({
                'Ticker': stock.ticker,
                'Quantity': stock.quantity,
                'Purchase Price': stock.purchase_price,
                'Current Price': stock.current_price,
                'Profit/Loss': stock.profit_loss(),
                'Change (%)': stock.percentage_change()
            })
        return summary

In [4]:
class StockPortfolioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Stock Portfolio Tracker")
        self.portfolio = Portfolio()

        self.setup_ui()

    def setup_ui(self):
        # Labels and entries
        tk.Label(self.root, text="Ticker:").grid(row=0, column=0)
        tk.Label(self.root, text="Quantity:").grid(row=1, column=0)
        tk.Label(self.root, text="Purchase Price:").grid(row=2, column=0)

        self.ticker_entry = tk.Entry(self.root)
        self.quantity_entry = tk.Entry(self.root)
        self.purchase_price_entry = tk.Entry(self.root)

        self.ticker_entry.grid(row=0, column=1)
        self.quantity_entry.grid(row=1, column=1)
        self.purchase_price_entry.grid(row=2, column=1)

        tk.Button(self.root, text="Add Stock", command=self.add_stock).grid(row=3, column=0, columnspan=2)
        tk.Button(self.root, text="Remove Stock", command=self.remove_stock).grid(row=4, column=0, columnspan=2)
        tk.Button(self.root, text="Show Summary", command=self.show_summary).grid(row=5, column=0, columnspan=2)

        self.summary_text = tk.Text(self.root, height=10, width=50)
        self.summary_text.grid(row=6, column=0, columnspan=2)

    def add_stock(self):
        ticker = self.ticker_entry.get()
        try:
            quantity = int(self.quantity_entry.get())
            purchase_price = float(self.purchase_price_entry.get())
            self.portfolio.add_stock(ticker, quantity, purchase_price)
            messagebox.showinfo("Success", f"Stock {ticker} added!")
        except ValueError:
            messagebox.showerror("Error", "Invalid input. Please enter valid quantity and purchase price.")

    def remove_stock(self):
        ticker = self.ticker_entry.get()
        self.portfolio.remove_stock(ticker)
        messagebox.showinfo("Success", f"Stock {ticker} removed!")

    def show_summary(self):
        summary = self.portfolio.portfolio_summary()
        self.summary_text.delete(1.0, tk.END)
        for stock in summary:
            self.summary_text.insert(tk.END, f"Ticker: {stock['Ticker']}\n")
            self.summary_text.insert(tk.END, f"Quantity: {stock['Quantity']}\n")
            self.summary_text.insert(tk.END, f"Purchase Price: ${stock['Purchase Price']:.2f}\n")
            self.summary_text.insert(tk.END, f"Current Price: ${stock['Current Price']:.2f}\n")
            self.summary_text.insert(tk.END, f"Profit/Loss: ${stock['Profit/Loss']:.2f}\n")
            self.summary_text.insert(tk.END, f"Change (%): {stock['Change (%)']:.2f}%\n")
            self.summary_text.insert(tk.END, "-"*40 + "\n")
        self.summary_text.insert(tk.END, f"\nTotal Portfolio Value: ${self.portfolio.portfolio_value():.2f}\n")
        self.summary_text.insert(tk.END, f"Total Profit/Loss: ${self.portfolio.total_profit_loss():.2f}\n")

In [6]:
if __name__ == "__main__":
    root = tk.Tk()
    app = StockPortfolioApp(root)
    root.mainloop()
